In [3]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("CrimePrediction").getOrCreate()

# Load dataset 
df = spark.read.parquet("C:/Users/ADMIN/DATA ANALYST/Projectdata/Chicago_Crimes_ZIP.parquet")
df.show(5)


+-------------+------------+--------+----------+--------------------+--------------------+----+-----------------+--------------------+-------------------+------+--------+----+--------+----+-------------+-------+-----------+-----------+----+--------------------+-------+
|            x|           y|      ID|CaseNumber|                Date|               Block|IUCR|      PrimaryType|         Description|LocationDescription|Arrest|Domestic|Beat|District|Ward|CommunityArea|FBICode|XCoordinate|YCoordinate|Year|           UpdatedOn|ZIPCode|
+-------------+------------+--------+----------+--------------------+--------------------+----+-----------------+--------------------+-------------------+------+--------+----+--------+----+-------------+-------+-----------+-----------+----+--------------------+-------+
|-87.677483468|41.955113932|10773691|  HZ539008|12/02/2016 06:15:...|  019XX W CUYLER AVE|1310|  CRIMINAL DAMAGE|         TO PROPERTY|          RESIDENCE| false|   false|1912|     019|  47| 

In [4]:
# Register DataFrame as a SQL temporary view
df.createOrReplaceTempView("crime_data")

# Query to select rows where 'Arrest' column is 'true' or 'false'
query = """
SELECT * 
FROM crime_data
WHERE Arrest IN ('true', 'false')
"""
filtered_df = spark.sql(query)
filtered_df.show(5)


+-------------+------------+--------+----------+--------------------+--------------------+----+-----------------+--------------------+-------------------+------+--------+----+--------+----+-------------+-------+-----------+-----------+----+--------------------+-------+
|            x|           y|      ID|CaseNumber|                Date|               Block|IUCR|      PrimaryType|         Description|LocationDescription|Arrest|Domestic|Beat|District|Ward|CommunityArea|FBICode|XCoordinate|YCoordinate|Year|           UpdatedOn|ZIPCode|
+-------------+------------+--------+----------+--------------------+--------------------+----+-----------------+--------------------+-------------------+------+--------+----+--------+----+-------------+-------+-----------+-----------+----+--------------------+-------+
|-87.677483468|41.955113932|10773691|  HZ539008|12/02/2016 06:15:...|  019XX W CUYLER AVE|1310|  CRIMINAL DAMAGE|         TO PROPERTY|          RESIDENCE| false|   false|1912|     019|  47| 

In [5]:
from pyspark.ml.feature import Tokenizer, HashingTF, StringIndexer
from pyspark.ml import Pipeline

# Combine 'PrimaryType' and 'Description' into one column (optional for better feature representation)
from pyspark.sql.functions import concat_ws
df = df.withColumn("text", concat_ws(" ", "PrimaryType", "Description"))

# Tokenization of 'text'
tokenizer = Tokenizer(inputCol="text", outputCol="words")

# Apply HashingTF to convert words into numeric features
hashing_tf = HashingTF(inputCol="words", outputCol="features")

# StringIndexer to convert 'Arrest' column to numeric label
indexer = StringIndexer(inputCol="Arrest", outputCol="label")

# Định nghĩa mô hình Logistic Regression
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(featuresCol="features", labelCol="label")

#Xây dựng Pipeline xử lý dữ liệu và huấn luyện
pipeline = Pipeline(stages=[tokenizer, hashing_tf, indexer, lr])

# Chia tập dữ liệu thành huấn luyện và kiểm tra, train_df: 80% dữ liệu dùng để huấn luyện.
# test_df: 20% dữ liệu dùng để đánh giá mô hình
train_df, test_df = df.randomSplit([0.8, 0.2], seed=1234)


In [6]:
# Fit the pipeline model
model = pipeline.fit(train_df)

# Make predictions on the test set
predictions = model.transform(test_df)

# Show predictions
predictions.select("PrimaryType", "Description", "Arrest", "label", "prediction").show(5)


+--------------------+----------------+----------------+-----+----------+
|         PrimaryType|     Description|          Arrest|label|prediction|
+--------------------+----------------+----------------+-----+----------+
|  DECEPTIVE PRACTICE| THEFT BY LESSEE|AIRPORT/AIRCRAFT|  8.0|       9.0|
|PUBLIC PEACE VIOL...|RECKLESS CONDUCT|            true|  1.0|       1.0|
|               THEFT|    RETAIL THEFT|            true|  1.0|       1.0|
|             BATTERY|          SIMPLE|            true|  1.0|       0.0|
|             BATTERY|          SIMPLE|           false|  0.0|       0.0|
+--------------------+----------------+----------------+-----+----------+
only showing top 5 rows



In [7]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Tạo các evaluator cho precision và recall
precision_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="precisionByLabel")
recall_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="recallByLabel")

# Compute precision and recall for class "1.0" (arrest == true)
precision = precision_evaluator.evaluate(predictions, {precision_evaluator.metricLabel: 1.0})
recall = recall_evaluator.evaluate(predictions, {recall_evaluator.metricLabel: 1.0})

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")


Precision: 0.8718
Recall: 0.5574


In [8]:
import time
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Start the timer
start_time = time.time()

# Train the model and make predictions
model = pipeline.fit(train_df)
predictions = model.transform(test_df)

# Use MulticlassClassificationEvaluator for precision and recall
precision_evaluator = MulticlassClassificationEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="precisionByLabel"
)
recall_evaluator = MulticlassClassificationEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="recallByLabel"
)

# Evaluate precision and recall for label 1.0 (Arrest == true)
precision = precision_evaluator.evaluate(predictions, {precision_evaluator.metricLabel: 1.0})
recall = recall_evaluator.evaluate(predictions, {recall_evaluator.metricLabel: 1.0})

# Calculate time taken
end_time = time.time()
total_time = end_time - start_time

# Output results
print(f"Total Time: {total_time:.2f} seconds")
print(f"Precision (label=1.0): {precision:.4f}")
print(f"Recall (label=1.0): {recall:.4f}")


Total Time: 154.21 seconds
Precision (label=1.0): 0.8718
Recall (label=1.0): 0.5574


In [9]:
def process_dataset(file_path, label):
    print(f"\nProcessing dataset: {label}")
    # Bắt đầu đo thời gian
    start_time = time.time()

    # Load data
    df = spark.read.parquet("C:/Users/ADMIN/DATA ANALYST/Projectdata/Chicago_Crimes_ZIP.parquet")

    # Chuẩn bị dữ liệu
    df = df.filter(df.Arrest.isin("true", "false"))
    df = df.withColumn("text", concat_ws(" ", "PrimaryType", "Description"))

    # Tokenizer, TF, Label Indexer
    tokenizer = Tokenizer(inputCol="text", outputCol="words")
    hashing_tf = HashingTF(inputCol="words", outputCol="features")
    indexer = StringIndexer(inputCol="Arrest", outputCol="label")
    lr = LogisticRegression(featuresCol="features", labelCol="label")

    # Pipeline
    pipeline = Pipeline(stages=[tokenizer, hashing_tf, indexer, lr])

    # Split
    train_df, test_df = df.randomSplit([0.8, 0.2], seed=42)

    # Train and predict
    model = pipeline.fit(train_df)
    predictions = model.transform(test_df)

    # Evaluate
    precision_eval = MulticlassClassificationEvaluator(
        labelCol="label", predictionCol="prediction", metricName="precisionByLabel"
    )
    recall_eval = MulticlassClassificationEvaluator(
        labelCol="label", predictionCol="prediction", metricName="recallByLabel"
    )
    precision = precision_eval.evaluate(predictions, {precision_eval.metricLabel: 1.0})
    recall = recall_eval.evaluate(predictions, {recall_eval.metricLabel: 1.0})

    # End time
    end_time = time.time()
    total_time = end_time - start_time

    # Print results
    print(f"Total Time for {label}: {total_time:.2f} seconds")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
process_dataset("C:/Users/ADMIN/DATA ANALYST/Projectdata/Chicago_Crimes_100k.csv", "100,000 rows")
process_dataset( "C:/Users/ADMIN/DATA ANALYST/Projectdata/Chicago_Crimes_10k.csv", "10,000 rows")


Processing dataset: 100,000 rows
Total Time for 100,000 rows: 10.27 seconds
Precision: 0.8846
Recall: 0.6765

Processing dataset: 10,000 rows
Total Time for 10,000 rows: 9.39 seconds
Precision: 0.8846
Recall: 0.6765
